<a href="https://colab.research.google.com/github/TakumaNakaguchi/Bayesian-of-cachexia/blob/main/%E8%A8%AA%E5%95%8F%E3%83%AA%E3%83%8F%E5%8A%B9%E6%9E%9C%E3%81%AE%E9%9A%8E%E5%B1%A4%E3%83%99%E3%82%A4%E3%82%BA%E3%83%A2%E3%83%87%E3%83%AB%E8%A7%A3%E6%9E%90%E3%82%B3%E3%83%BC%E3%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# ライブラリのインストール（Google Colabで初回実行時のみ必要）
# =============================================================================
# !pip install pymc==5.10.3 arviz==0.17.0 pandas==2.1.4 numpy==1.26.2 matplotlib==3.8.2 seaborn==0.13.1

# =============================================================================
# ライブラリのインポート
# =============================================================================
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

print(f"PyMC Version: {pm.__version__}")
print(f"ArviZ Version: {az.__version__}")

# =============================================================================
# Part 1: 仮想データの生成と準備
# =============================================================================
# 論文の記述に基づき、仮想データを生成します。
# 実際のデータを使用する場合は、このセクションをコメントアウトし、
# 次の「実際のデータ読み込み」セクションを使用してください。

# シードを設定して再現性を確保
np.random.seed(42)

# 対象者数
n_patients = 49

# 論文の表1、表2に基づき、各変数の関係性を考慮して仮想データを生成
# 相関行列を定義（探索的解析の結果を参考に設定）
# Variables: age, bmi, grip, ftsst, crp, mmse, bi_initial, bi_gain
corr_matrix = np.array([
    [1.00, 0.10, -0.30, 0.40, 0.30, -0.30, -0.20, -0.25], # age
    [0.10, 1.00, 0.20, 0.10, 0.15, -0.10, 0.10, -0.21], # bmi
    [0.10, 0.20, 1.00, -0.30, -0.20, 0.20, 0.30, 0.10], # grip
    [0.40, 0.10, -0.30, 1.00, 0.25, -0.40, -0.30, -0.22], # ftsst
    [0.30, 0.15, -0.20, 0.25, 1.00, -0.20, -0.20, -0.19], # crp
    [-0.30, -0.10, 0.20, -0.40, -0.20, 1.00, 0.30, 0.18], # mmse
    [-0.20, 0.10, 0.30, -0.30, -0.20, 0.30, 1.00, -0.10], # bi_initial
    [-0.25, -0.21, 0.10, -0.22, -0.19, 0.18, -0.10, 1.00]  # bi_gain
])

# 平均と標準偏差
means = np.array([84.5, 19.9, 17.0, 35.8, 3.26, 24.8, 67.7, 13.8]) # bi_gainの平均はbi_6m - bi_initialから算出
stds = np.array([7.8, 4.7, 4.0, 8.4, 6.89, 4.8, 16.1, 15.0]) # bi_gainのSDは仮定

# 多変量正規分布に従うデータを生成
data = np.random.multivariate_normal(means, np.diag(stds) @ corr_matrix @ np.diag(stds), n_patients)

df = pd.DataFrame(data, columns=['age', 'bmi', 'grip', 'ftsst', 'crp', 'mmse', 'bi_initial', 'bi_gain_temp'])

# カテゴリ変数の生成
# 性別 (女性=1, 男性=0): 75.5%が女性
df['sex'] = np.random.choice([1, 0], n_patients, p=[0.755, 0.245])
# 訪問リハ頻度 (週2回以上=1, 週1回=0): 40.9%が週2回以上
df['rehab_freq'] = np.random.choice([1, 0], n_patients, p=[0.409, 0.591])
# 住居環境 (施設=1, 自宅=0): 44.9%が施設
df['residence'] = np.random.choice([1, 0], n_patients, p=[0.449, 0.551])

# 生成したデータの値を論文の範囲に収めるように調整
df['age'] = np.clip(df['age'], 65, 100).astype(int)
df['bmi'] = np.clip(df['bmi'], 12, 30)
df['grip'] = np.clip(df['grip'], 5, 30)
df['ftsst'] = np.clip(df['ftsst'], 15, 60)
df['crp'] = np.clip(df['crp'], 0.1, 30)
df['mmse'] = np.clip(df['mmse'], 10, 30).astype(int)
df['bi_initial'] = np.clip(df['bi_initial'], 0, 100).astype(int)
df['bi_6months'] = np.clip(df['bi_initial'] + df['bi_gain_temp'], 0, 100).astype(int)
df['bi_gain'] = df['bi_6months'] - df['bi_initial']

# 不要な列を削除
df = df.drop(columns=['bi_gain_temp'])

print("--- 生成された仮想データの基本統計量 ---")
print(df.describe())
print("\n--- カテゴリ変数の分布 ---")
print(df[['sex', 'rehab_freq', 'residence']].apply(pd.Series.value_counts))


# =============================================================================
# 実際のデータ読み込み（このセクションはコメントアウトされています）
# =============================================================================
# # CSVファイルなどから実際のデータを読み込む場合は、以下のコードを使用してください。
# # ファイルパスは適宜変更してください。
# try:
#     df = pd.read_csv('your_data.csv')
#     # 論文に合わせて列名を変更
#     # df.columns = ['age', 'sex', 'bmi', 'rehab_freq', 'residence',
#     #               'bi_initial', 'bi_6months', 'grip', 'ftsst', 'crp', 'mmse']
#     # BI利得を計算
#     # df['bi_gain'] = df['bi_6months'] - df['bi_initial']
#     print("--- 実際のデータを読み込みました ---")
# except FileNotFoundError:
#     print("--- your_data.csv が見つかりません。仮想データを引き続き使用します。 ---")


# =============================================================================
# データの前処理：変数の標準化
# =============================================================================
# 連続変数を標準化（平均0, 標準偏差1）します。
# これにより、モデルの収束が改善され、係数の解釈が容易になります。
continuous_vars = ['age', 'bmi', 'grip', 'ftsst', 'crp', 'mmse', 'bi_initial']
categorical_vars = ['sex', 'rehab_freq', 'residence']

# 標準化前の統計量を保存
means_std = df[continuous_vars].mean()
stds_std = df[continuous_vars].std()

# 標準化
df_std = df.copy()
for var in continuous_vars:
    df_std[var] = (df[var] - means_std[var]) / stds_std[var]

print("\n--- データの前処理が完了しました ---")


# =============================================================================
# Part 2: 主解析 - 階層ベイズモデル
# =============================================================================
# 論文の主解析モデルを構築し、MCMCサンプリングを実行します。
# 従属変数: BI利得 (bi_gain)
# 説明変数: bi_initial, age, sex, bmi, rehab_freq, residence, grip, ftsst, crp, mmse

coords = {"coeffs": ['bi_initial', 'age', 'sex', 'bmi', 'rehab_freq', 'residence', 'grip', 'ftsst', 'crp', 'mmse']}

with pm.Model(coords=coords) as main_model:
    # --- 事前分布の設定 ---
    # 切片 (Intercept): 先行研究に基づき、平均5.8, 標準偏差5.33の正規分布
    intercept = pm.Normal('intercept', mu=5.8, sigma=5.33)

    # 係数 (beta): 論文では探索的解析に基づいて情報事前分布と弱情報事前分布を使い分けていますが、
    # ここでは一般的に使われる弱情報事前分布（平均0、標準偏差を広めにとった正規分布）を設定します。
    # これにより、データが持つ情報をより強く反映させることができます。
    betas = pm.Normal('betas', mu=0, sigma=10, dims="coeffs")

    # モデルの誤差 (sigma): 半コーシー分布
    sigma = pm.HalfCauchy('sigma', beta=5)

    # --- 線形モデルの定義 ---
    mu = (intercept +
          betas[0] * df_std['bi_initial'] +
          betas[1] * df_std['age'] +
          betas[2] * df_std['sex'] +
          betas[3] * df_std['bmi'] +
          betas[4] * df_std['rehab_freq'] +
          betas[5] * df_std['residence'] +
          betas[6] * df_std['grip'] +
          betas[7] * df_std['ftsst'] +
          betas[8] * df_std['crp'] +
          betas[9] * df_std['mmse']
         )

    # --- 尤度 ---
    # BI利得が正規分布に従うと仮定
    bi_gain_obs = pm.Normal('bi_gain_obs', mu=mu, sigma=sigma, observed=df['bi_gain'])

    # --- MCMCサンプリングの実行 ---
    # 論文の設定に合わせて、4チェーン、2000反復（うち500をバーンイン）
    print("\n--- 主解析モデルのサンプリングを開始します ---")
    trace_main = pm.sample(2000, tune=500, chains=4, target_accept=0.9, random_seed=42)
    print("--- サンプリングが完了しました ---")

# --- 結果の表示 ---
print("\n--- 主解析モデルの結果 ---")
summary_main = az.summary(trace_main, var_names=['intercept', 'betas', 'sigma'], hdi_prob=0.95, round_to=2)
print(summary_main)

# 収束診断 (R-hatが1.1未満であれば良好)
if (summary_main['r_hat'] > 1.1).any():
    print("\n警告: いくつかのパラメータで収束に問題がある可能性があります (R-hat > 1.1)。")
else:
    print("\nすべてのパラメータで収束を確認しました (R-hat < 1.1)。")


# =============================================================================
# Part 3: 層別解析 - 訪問リハビリテーション頻度別のBI利得
# =============================================================================
# 主解析モデルの結果を用いて、訪問リハ頻度別のBI利得の事後分布を計算・可視化します。

# 他の変数を平均値（標準化後データでは0）に固定した場合のBI利得を計算
posterior = trace_main.posterior
intercept_samples = posterior['intercept'].values.flatten()
rehab_freq_beta_samples = posterior['betas'].sel(coeffs='rehab_freq').values.flatten()

# 週1回群 (rehab_freq=0) のBI利得
# rehab_freq=0は標準化すると負の値になるため、標準化前の値で計算
rehab_freq_0_std = (0 - means_std['rehab_freq']) / stds_std['rehab_freq']
gain_weekly = intercept_samples + rehab_freq_beta_samples * rehab_freq_0_std

# 週2回以上群 (rehab_freq=1) のBI利得
rehab_freq_1_std = (1 - means_std['rehab_freq']) / stds_std['rehab_freq']
gain_biweekly = intercept_samples + rehab_freq_beta_samples * rehab_freq_1_std

print("\n--- 層別解析の結果 ---")
print(f"週1回群のBI利得（事後平均）: {gain_weekly.mean():.2f} (95%信用区間: {np.quantile(gain_weekly, 0.025):.2f} - {np.quantile(gain_weekly, 0.975):.2f})")
print(f"週2回以上群のBI利得（事後平均）: {gain_biweekly.mean():.2f} (95%信用区間: {np.quantile(gain_biweekly, 0.025):.2f} - {np.quantile(gain_biweekly, 0.975):.2f})")

# --- 結果の可視化 ---
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(10, 6))
sns.kdeplot(gain_weekly, ax=ax, label='Weekly Rehab (週1回)', fill=True)
sns.kdeplot(gain_biweekly, ax=ax, label='Bi-weekly+ Rehab (週2回以上)', fill=True)
ax.set_title('Posterior Distribution of BI Gain by Rehab Frequency (訪問リハ頻度別BI利得の事後分布)', fontsize=16)
ax.set_xlabel('Estimated BI Gain (推定BI利得)', fontsize=12)
ax.set_ylabel('Density (密度)', fontsize=12)
ax.legend()
plt.show()


# =============================================================================
# Part 4: 感度解析
# =============================================================================
# 論文で述べられている3つの感度解析を実施し、結果の頑健性を確認します。

# --- ① BI利得の事前分布変更 ---
# 切片の事前分布の標準偏差をMDCである16.3に変更
with pm.Model(coords=coords) as sensitivity_model_1:
    intercept = pm.Normal('intercept', mu=5.8, sigma=16.3) # 標準偏差を変更
    betas = pm.Normal('betas', mu=0, sigma=10, dims="coeffs")
    sigma = pm.HalfCauchy('sigma', beta=5)
    mu = (intercept + pm.math.dot(df_std[coords['coeffs']].values, betas))
    bi_gain_obs = pm.Normal('bi_gain_obs', mu=mu, sigma=sigma, observed=df['bi_gain'])

    print("\n--- 感度解析① (事前分布変更) のサンプリングを開始します ---")
    trace_sens_1 = pm.sample(2000, tune=500, chains=4, random_seed=42)
    print("--- サンプリングが完了しました ---")

print("\n--- 感度解析① の結果 ---")
print(az.summary(trace_sens_1, var_names=['intercept', 'betas', 'sigma'], hdi_prob=0.95, round_to=2))


# --- ② 総因果効果の評価 ---
# 従属変数を6ヶ月後BI、説明変数から開始時BIを除外
coords_total = {"coeffs": ['age', 'sex', 'bmi', 'rehab_freq', 'residence', 'grip', 'ftsst', 'crp', 'mmse']}
with pm.Model(coords=coords_total) as sensitivity_model_2:
    intercept = pm.Normal('intercept', mu=df['bi_6months'].mean(), sigma=10)
    betas = pm.Normal('betas', mu=0, sigma=10, dims="coeffs")
    sigma = pm.HalfCauchy('sigma', beta=5)
    mu = (intercept + pm.math.dot(df_std[coords_total['coeffs']].values, betas))
    bi_6m_obs = pm.Normal('bi_6m_obs', mu=mu, sigma=sigma, observed=df['bi_6months'])

    print("\n--- 感度解析② (総因果効果) のサンプリングを開始します ---")
    trace_sens_2 = pm.sample(2000, tune=500, chains=4, random_seed=42)
    print("--- サンプリングが完了しました ---")

print("\n--- 感度解析② の結果 (訪問リハ頻度の効果に注目) ---")
print(az.summary(trace_sens_2, var_names=['betas'], hdi_prob=0.95, round_to=2).loc[f"betas[{'rehab_freq'}]"])


# --- ③ 直接因果効果の評価 ---
# 従属変数を6ヶ月後BI、説明変数に開始時BIを追加
coords_direct = {"coeffs": ['bi_initial', 'age', 'sex', 'bmi', 'rehab_freq', 'residence', 'grip', 'ftsst', 'crp', 'mmse']}
with pm.Model(coords=coords_direct) as sensitivity_model_3:
    intercept = pm.Normal('intercept', mu=df['bi_6months'].mean(), sigma=10)
    betas = pm.Normal('betas', mu=0, sigma=10, dims="coeffs")
    sigma = pm.HalfCauchy('sigma', beta=5)
    mu = (intercept + pm.math.dot(df_std[coords_direct['coeffs']].values, betas))
    bi_6m_obs = pm.Normal('bi_6m_obs', mu=mu, sigma=sigma, observed=df['bi_6months'])

    print("\n--- 感度解析③ (直接因果効果) のサンプリングを開始します ---")
    trace_sens_3 = pm.sample(2000, tune=500, chains=4, random_seed=42)
    print("--- サンプリングが完了しました ---")

print("\n--- 感度解析③ の結果 (訪問リハ頻度の効果に注目) ---")
print(az.summary(trace_sens_3, var_names=['betas'], hdi_prob=0.95, round_to=2).loc[f"betas[{'rehab_freq'}]"])